In [ ]:
from openai import OpenAI
import json
import os
import random
import tqdm
import tiktoken

api_key = ""
client = OpenAI(api_key=api_key)
data_dir = "../../data/xsum.raw_data.json"
dataset=data_dir.split("/")[-1].split(".")[0]
model="gpt-4o"

with open(data_dir,"r") as f:
    sentences = json.load(f)['original']

    
result = dict(original=sentences,rewritten=[])

for idx, sentence in enumerate(tqdm.tqdm(sentences)):
    encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(sentence)[:30]
    decode_sentence = encoding.decode(encoded)
    # print(decode_sentence)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"You are a News writer."},
            {"role": "user", "content": f"Please write an article with about 150 words starting exactly with: {decode_sentence}"}
        ],
        temperature=0.7
    )
    result['rewritten'].append(completion.choices[0].message.content)

with open(f'../../data/{model}/{dataset}_generation_{model}.raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)